In [2]:
import os

cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /Users/tarasbohdan/Desktop/Experiments-AAAI-24


In [3]:
%matplotlib inline

import sys
sys.path.append("../")
import copy

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
from aif360.algorithms.preprocessing.lfr import LFR

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
import numpy as np


In [4]:
class Config:
  file_name = './data/student_performance/student-mat.csv'

In [5]:
from notebooks.mult_repair_levels_student_performance.StudentPerformanceDataset import StudentPerformanceDataset

data_loader = StudentPerformanceDataset(Config.file_name)
data_loader.X_data.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,Mjob,Fjob,reason,guardian,sex
0,18,4,4,2,2,0,4,3,4,1,1,3,6,0,0,at_home,teacher,course,mother,F
1,17,1,1,1,2,0,5,3,3,1,1,3,4,0,0,at_home,other,course,father,F
2,15,1,1,1,2,3,4,3,2,2,3,3,10,0,0,at_home,other,other,mother,F
3,15,4,2,1,3,0,3,2,2,1,1,5,2,1,1,health,services,home,mother,F
4,16,3,3,1,2,0,4,3,2,1,2,5,4,0,1,other,other,home,father,F


In [6]:
init_data_loader = copy.deepcopy(data_loader)

In [7]:
from virny.preprocessing import preprocess_dataset
from source.preprocessing import get_simple_preprocessor

data_loader.categorical_columns = [col for col in data_loader.categorical_columns if
                                   col not in ('sex')]
data_loader.X_data['sex_binary'] = data_loader.X_data['sex'].apply(lambda x: 1 if x == 'M' else 0)
data_loader.full_df = data_loader.full_df.drop(['sex'], axis=1)
data_loader.X_data = data_loader.X_data.drop(['sex'], axis=1)

# Preprocess the dataset using the defined preprocessor
column_transformer = get_simple_preprocessor(data_loader)
base_flow_dataset = preprocess_dataset(data_loader, column_transformer, 0.3, 42)
base_flow_dataset.init_features_df = init_data_loader.full_df.drop(init_data_loader.target, axis=1,
                                                                   errors='ignore')
base_flow_dataset.X_train_val['sex_binary'] = data_loader.X_data.loc[
    base_flow_dataset.X_train_val.index, 'sex_binary']
base_flow_dataset.X_test['sex_binary'] = data_loader.X_data.loc[base_flow_dataset.X_test.index, 'sex_binary']

/var/folders/cg/l243p36n10s5v1x_vrt2wzs80000gn/T/ipykernel_68565/2079174263.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_loader.X_data['sex_binary'] = data_loader.X_data['sex'].apply(lambda x: 1 if x == 'M' else 0)
/Users/tarasbohdan/Desktop/Experiments-AAAI-24/virny_venv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [8]:
from sklearn.model_selection import train_test_split

full_df = data_loader.X_data

dataset_orig_train, dataset_orig_test = train_test_split(full_df, test_size=0.3)

In [9]:
sensitive_attribute = 'sex_binary'

In [10]:
train_df = base_flow_dataset.X_train_val
train_df[base_flow_dataset.target] = base_flow_dataset.y_train_val

test_df = base_flow_dataset.X_test
test_df[base_flow_dataset.target] = base_flow_dataset.y_test

In [11]:
train_binary_dataset = BinaryLabelDataset(df=train_df,
                                              label_names=[data_loader.target],
                                              protected_attribute_names=[sensitive_attribute],
                                              favorable_label=1,
                                              unfavorable_label=0)
test_binary_dataset = BinaryLabelDataset(df=test_df,
                                         label_names=[data_loader.target],
                                         protected_attribute_names=[sensitive_attribute],
                                         favorable_label=1,
                                         unfavorable_label=0)

In [12]:
# Metric for the original dataset
privileged_groups = [{'sex_binary': 1}]
unprivileged_groups = [{'sex_binary': 0}]

metric_orig_train = BinaryLabelDatasetMetric(train_binary_dataset,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())
metric_orig_test = BinaryLabelDatasetMetric(test_binary_dataset,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original test dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.034286


#### Original test dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.154042


In [28]:
TR = LFR(unprivileged_groups=unprivileged_groups,
         privileged_groups=privileged_groups,
         k=3, Ax=0.1, Ay=0.5, Az=4.0,
         verbose=1
        )
TR = TR.fit(train_binary_dataset, maxiter=5000, maxfun=5000)

step: 0, loss: 0.4809912317171258, L_x: 1.3810962672398244,  L_y: 0.6210111392736323,  L_z: 0.008094008839081809
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          102     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.80991D-01    |proj g|=  1.19131D-01
step: 250, loss: 0.4545641387297948, L_x: 1.3801652538352993,  L_y: 0.6194608197858771,  L_z: 0.0017043008633315833

At iterate    1    f=  4.54564D-01    |proj g|=  1.28906D-01
step: 500, loss: 0.4574143723247141, L_x: 1.3793609912655986,  L_y: 0.6186717380447676,  L_z: 0.002535601043942607

At iterate    2    f=  4.51204D-01    |proj g|=  1.16283D-01

At iterate    3    f=  4.49093D-01    |proj g|=  1.28476D-01
step: 750, loss: 0.4495155088083656, L_x: 1.3795554551067073,  L_y: 0.6189619372896487,  L_z: 0.000519748663217634

At iterate    4    f=  4.48418D-01    |proj g|=  1.17345D-01
step: 1000, loss: 0.4540590217875803, L_x: 1.3797901915440722,  

In [29]:
dataset_transf_train = TR.transform(train_binary_dataset)
dataset_transf_test = TR.transform(test_binary_dataset)

In [30]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())
metric_transf_test = BinaryLabelDatasetMetric(dataset_transf_test,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Transformed test dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_test.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.029206


#### Transformed test dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.045506


In [31]:
train_df, _ = train_binary_dataset.convert_to_dataframe()
test_df, _ = test_binary_dataset.convert_to_dataframe()

In [32]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=42)

In [33]:
train_X = train_df.drop(['G3'], axis=1)
train_y = train_df['G3']

In [34]:
test_X = test_df.drop(['G3'], axis=1)
test_y = test_df['G3']

In [35]:
rfc = rfc.fit(train_X, train_y)

In [36]:
y_pred = rfc.predict(test_X)

In [37]:
report = classification_report(test_y, y_pred)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

         0.0       0.85      0.96      0.90        46
         1.0       0.97      0.89      0.93        73

    accuracy                           0.92       119
   macro avg       0.91      0.92      0.91       119
weighted avg       0.92      0.92      0.92       119



In [38]:
train_df, _ = dataset_transf_train.convert_to_dataframe()
test_df, _ = dataset_transf_test.convert_to_dataframe()


rfc = RandomForestClassifier(random_state=42)

train_X = train_df.drop(['G3'], axis=1)
train_y = train_df['G3']

test_X = test_df.drop(['G3'], axis=1)
test_y = test_df['G3']

rfc = rfc.fit(train_X, train_y)

y_pred = rfc.predict(test_X)

report = classification_report(test_y, y_pred)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        28
         1.0       1.00      1.00      1.00        91

    accuracy                           1.00       119
   macro avg       1.00      1.00      1.00       119
weighted avg       1.00      1.00      1.00       119



### Tune parameters for LFR

In [24]:
TR = LFR(unprivileged_groups=unprivileged_groups,
         privileged_groups=privileged_groups,
         k=1, Ax=0.1, Ay=0.1, Az=5,
         verbose=1
        )
TR = TR.fit(train_binary_dataset, maxiter=5000, maxfun=5000)

dataset_transf_train = TR.transform(train_binary_dataset)
dataset_transf_test = TR.transform(test_binary_dataset)

train_df, _ = dataset_transf_train.convert_to_dataframe()
test_df, _ = dataset_transf_test.convert_to_dataframe()


rfc = RandomForestClassifier(random_state=42)

train_X = train_df.drop(['G3'], axis=1)
train_y = train_df['G3']

test_X = test_df.drop(['G3'], axis=1)
test_y = test_df['G3']

rfc = rfc.fit(train_X, train_y)

y_pred = rfc.predict(test_X)

report = classification_report(test_y, y_pred)
print("Classification Report:\n", report)

step: 0, loss: 0.22743167378713616, L_x: 1.58350173279866,  L_y: 0.6908150050727015,  L_z: 0.0
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           34     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.27432D-01    |proj g|=  1.21512D-01

At iterate    1    f=  2.18864D-01    |proj g|=  1.67716D-02

At iterate    2    f=  2.16821D-01    |proj g|=  1.02379D-02

At iterate    3    f=  2.08828D-01    |proj g|=  4.13474D-02

At iterate    4    f=  1.99629D-01    |proj g|=  7.13771D-02

At iterate    5    f=  1.85753D-01    |proj g|=  8.44147D-02

At iterate    6    f=  1.71273D-01    |proj g|=  3.54195D-02
step: 250, loss: 0.1696971272844709, L_x: 1.0823854703358897,  L_y: 0.614585802508819,  L_z: 0.0

At iterate    7    f=  1.69697D-01    |proj g|=  2.76917D-03

At iterate    8    f=  1.69671D-01    |proj g|=  4.16873D-05

At iterate    9    f=  1.69670D-01    |proj g|=  2.17361D-05

At iterate   10    f

In [25]:
from aif360.algorithms.preprocessing import LFR
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# k - Number of prototypes
# Input recontruction quality - Ax
# Fairness constraint - Az
# Output prediction error - Ay
def find_optimal():

    k_values = [1, 2, 3, 4, 5]
    Az_values = [1.0, 5.0, 25.0, 50.0]
    Ay_values = [1.0, 2.0, 3.0, 4.0, 10.0]
    Ax_values = [0.01, 0.1, 1.0, 10.0]

    best_accuracy = 0.0
    best_params = None

    for k in k_values:
        for Az in Az_values:
            for Ay in Ay_values:
                for Ax in Ax_values:
                    lfr = LFR(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups,
                              k=k, Az=Az, Ay=Ay, Ax=Ax)

                    lfr = lfr.fit(train_binary_dataset)

                    dataset_transf_train = lfr.transform(train_binary_dataset)

                    train_df, _ = dataset_transf_train.convert_to_dataframe()

                    train_X = train_df.drop(['G3'], axis=1)
                    train_y = train_df['G3']

                    rfc = RandomForestClassifier(random_state=42)

                    rfc.fit(train_X, train_y)

                    dataset_transf_test = lfr.transform(test_binary_dataset)
                    test_df, _ = dataset_transf_test.convert_to_dataframe()
                    test_X = test_df.drop(['G3'], axis=1)
                    test_y = test_df['G3']
                    y_pred = rfc.predict(test_X)

                    accuracy = accuracy_score(test_y, y_pred)

                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_params = (k, Az, Ay, Ax)

                    if accuracy == 1.0:
                        return best_accuracy, best_params

    return best_accuracy, best_params

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


In [26]:
best_accuracy, best_params = find_optimal()
print("Best Parameters (k, Az, Ay, Ax):", best_params)
print("Best Accuracy:", best_accuracy)

Best Parameters (k, Az, Ay, Ax): (1, 1.0, 1.0, 0.01)
Best Accuracy: 1.0
